In [6]:
import openpyxl
import mne
import mnelab
import numpy
import matplotlib
import xgboost
import pyriemann
import scipy
import pickle
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# 行列标识符,P300二分类转字符36分类用
chars = {'A': (1, 7), 'B': (1, 8), 'C': (1, 9), 'D': (1, 10), 'E': (1, 11), 'F': (1, 12), 
         'G': (2, 7), 'H': (2, 8), 'I': (2, 9), 'J': (2, 10), 'K': (2, 11), 'L': (2, 12), 
         'M': (3, 7), 'N': (3, 8), 'O': (3, 9), 'P': (3, 10), 'Q': (3, 11), 'R': (3, 12), 
         'S': (4, 7), 'T': (4, 8), 'U': (4, 9), 'V': (4, 10), 'W': (4, 11), 'X': (4, 12), 
         'Y': (5, 7), 'Z': (5, 8), '1': (5, 9), '2': (5, 10), '3': (5, 11), '4': (5, 12), 
         '5': (6, 7), '6': (6, 8), '7': (6, 9), '8': (6, 10), '9': (6, 11), '0': (6, 12), 
        }

In [32]:
# 加载本地数据集
dataset_s1 = pd.read_csv('../data/s1_train.csv')
dataset_s2 = pd.read_csv('../data/s2_train.csv')
dataset_s3 = pd.read_csv('../data/s3_train.csv')
dataset_s4 = pd.read_csv('../data/s4_train.csv')
dataset_s5 = pd.read_csv('../data/s5_train.csv')
dataset = np.concatenate((dataset_s1, dataset_s2, dataset_s3, dataset_s4, dataset_s5), axis=0)

# 随机删除过多的负样本,使正负样本1:1
delete_num = 0
while delete_num<2400:
    index = np.random.choice(dataset.shape[0],1,replace=True)
    # 只删负样本
    if np.squeeze(dataset[index])[-1] == 0.0:
        dataset = np.delete(dataset, index, axis=0)
        delete_num += 1

print(dataset.shape[0])
data = dataset[:, 1:521]
label = dataset[:, 521]
for index in range(len(label)):
    if label[index] != 0:
        label[index] = 1

print(data.shape, len(label))

# 划分数据集
data_train, data_eval, label_train, label_eval = train_test_split(data, label, test_size=0.2, random_state=1)

1200
(1200, 520) 1200


In [22]:
# SVM二分类器
from sklearn.svm import SVC
clf_svm = SVC(kernel='rbf', probability=True)
clf_svm.fit(data_train, label_train)
file_svm = open('model/svm.pickle', 'wb')
clf_svm_model = pickle.dump(clf_svm, file_svm)
pred_svm = clf_svm.predict(data_eval)
num1 = 0
num2 = 0
for i in pred_svm:
    if i == 0:
        num2 += 1
    else:
        num1 += 1
print(num1, num2)
print(classification_report(label_eval, pred_svm))
print("Accuracy: ", accuracy_score(label_eval,pred_svm))

train finished
0 240
              precision    recall  f1-score   support

         0.0       0.47      1.00      0.64       113
         1.0       0.00      0.00      0.00       127

    accuracy                           0.47       240
   macro avg       0.24      0.50      0.32       240
weighted avg       0.22      0.47      0.30       240

Accuracy:  0.4708333333333333


/home/chris/anaconda3/envs/lyft/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
# 随机森林二分类器
clf_RF = RandomForestClassifier(n_estimators=10,)
clf_RF.fit(data_train, label_train)
file_RF = open('model/RF.pickle', 'wb')
clf_RF_model = pickle.dump(clf_RF, file_RF)
pred_RF = clf_RF.predict(data_eval)
print(classification_report(label_eval, pred_RF))
print("Accuracy: ", accuracy_score(label_eval,pred_RF))
num1 = 0
num2 = 0
for i in pred_RF:
    if i == 0:
        num2 += 1
    else:
        num1 += 1
print(num1, num2)

              precision    recall  f1-score   support

         0.0       0.48      0.52      0.50       122
         1.0       0.45      0.42      0.43       118

    accuracy                           0.47       240
   macro avg       0.47      0.47      0.46       240
weighted avg       0.47      0.47      0.47       240

Accuracy:  0.4666666666666667
108 132


In [28]:
# xgboost二分类器
import xgboost
clf_xgboost = xgboost.XGBClassifier()
clf_xgboost.fit(data_train, label_train)
file_xgboost = open('model/xgboost1.pickle', 'wb')
clf_xgboost_model = pickle.dump(clf_xgboost, file_xgboost)
pred_xgboost = clf_xgboost.predict(data_eval)
print(classification_report(label_eval, pred_xgboost))
print("Accuracy: ", accuracy_score(label_eval,pred_xgboost))
num1 = 0
num2 = 0
for i in pred_xgboost:
    if i == 0:
        num2 += 1
    else:
        num1 += 1
print(num1, num2)

train finished
              precision    recall  f1-score   support

         0.0       0.50      0.53      0.52       118
         1.0       0.52      0.49      0.51       122

    accuracy                           0.51       240
   macro avg       0.51      0.51      0.51       240
weighted avg       0.51      0.51      0.51       240

Accuracy:  0.5125
115 125


In [ ]:
# 字符识别
test_data_s1 = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S1/S1_test_data.xlsx')
test_data_s1 = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S1/S1_test_event.xlsx')